In [1]:
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import json

In [2]:
# Charger le fichier Excel
file_path = 'Résultats élections.xlsx'
excel_data = pd.ExcelFile(file_path)

# Charger la feuille "Résultats Complets"
df_results = pd.read_excel(file_path, sheet_name='Résultats Complets')
df_results = df_results.dropna(axis=1, how='all')


In [3]:
# Identifier les colonnes de base et les variables d'intérêt
columns_to_keep = ['Circonscription', 'bdv']
columns_of_interest = ['Inscrits', 'Abstentions', 'Votants', 'Votans_emargement', 'Blancs', 'Nuls', 'Exprimés']

# Identifier toutes les colonnes relatives aux élections
election_columns = [col for col in df_results.columns if any(var in col for var in columns_of_interest)]
print(election_columns)
# Extraire les identifiants d'élection
election_identifiers = set(col.split('_')[0] for col in election_columns if '_' in col)
print(election_identifiers)

['REG21_t1_Inscrits', 'REG21_t1_Abstentions', 'REG21_t1_Votants', 'REG21_t1_Votants_emargement', 'REG21_t1_Blancs', 'REG21_t1_Nuls', 'REG21_t1_Exprimés', 'REG21_t2_Inscrits', 'REG21_t2_Abstentions', 'REG21_t2_Votants', 'REG21_t2_Votants_emargement', 'REG21_t2_Blancs', 'REG21_t2_Nuls', 'REG21_t2_Exprimés', 'PRE22_t1_Inscrits', 'PRE22_t1_Abstentions', 'PRE22_t1_Votants', 'PRE22_t1_Blancs', 'PRE22_t1_Nuls', 'PRE22_t1_Exprimés', 'PRE22_t2_Inscrits', 'PRE22_t2_Abstentions', 'PRE22_t2_Votants', 'PRE22_t2_Votants_emargement', 'PRE22_t2_Blancs', 'PRE22_t2_Nuls', 'PRE22_t2_Exprimés', 'LEG22_t1_Inscrits', 'LEG22_t1_Abstentions', 'LEG22_t1_Votants', 'LEG22_t1_Blancs', 'LEG22_t1_Nuls', 'LEG22_t1_Exprimés', 'LEG22_t2_Inscrits', 'LEG22_t2_Abstentions', 'LEG22_t2_Votants', 'LEG22_t2_Blancs', 'LEG22_t2_Nuls', 'LEG22_t2_Exprimés', 'EUR24_t1_Inscrits', 'EUR24_t1_Abstentions', 'EUR24_t1_Votants', 'EUR24_t1_Blancs', 'EUR24_t1_Nuls', 'EUR24_t1_Exprimés', 'LEG24_t1_Inscrits', 'LEG24_t1_Abstentions', 'LEG24_

In [4]:
# Créer un dictionnaire pour stocker les dataframes par élection
election_dataframes = {}

# Boucle pour traiter chaque élection
for election in election_identifiers:
    # Identifier les colonnes spécifiques à cette élection
    election_columns = [col for col in df_results.columns if col.startswith(election)]
    
    # Combiner avec les colonnes de base
    subset_columns = columns_to_keep + election_columns
    
    # Filtrer le dataframe
    df_filtered = df_results[subset_columns].copy()
    
    # Standardiser les noms de colonnes
    standardized_columns = columns_to_keep + [col.split('_', 1)[1] for col in election_columns]
    df_filtered.columns = standardized_columns
    
    # Sauvegarder le dataframe dans le dictionnaire
    election_dataframes[election] = df_filtered

In [5]:
# Ajuster les dataframes pour inclure une colonne "Tour"
adjusted_election_dataframes = {}

for election, df in election_dataframes.items():
    # Transformer en format long pour gérer les tours
    id_vars = columns_to_keep  # Colonnes de base
    value_vars = [col for col in df.columns if col not in columns_to_keep]
    
    # Ajouter la colonne 'Tour' et nettoyer les noms de variables
    df_long = df.melt(id_vars=id_vars, var_name='Variable', value_name='Valeur')
    df_long['Tour'] = df_long['Variable'].str.extract(r't(\d)').astype(int)
    df_long['Variable'] = df_long['Variable'].str.replace(r't\d_', '', regex=True)
    
    # Revenir au format large avec la colonne 'Tour'
    df_wide = df_long.pivot_table(index=id_vars + ['Tour'], columns='Variable', values='Valeur').reset_index()
    
    # Sauvegarder le dataframe ajusté
    adjusted_election_dataframes[election] = df_wide

In [6]:
adjusted_election_dataframes.keys()

dict_keys(['LEG24', 'PRE22', 'REG21', 'LEG22', 'EUR24'])

In [7]:
adjusted_election_dataframes['EUR24']

Variable,Circonscription,bdv,Tour,Abstentions,Alliance rurale,Asselineau,Besoin europe,Blancs,Changer europe,Democratie representative,...,Parti des travailleurs,Parti pirate,Pour une autre europe,Pour une democratie reelle,Prenons nous en main,RN,Reveiller europe,Unite nationale,Urgence revolution,Votants
0,1.0,5,1,350.0,6.0,3.0,91.0,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,38.0,80.0,0.0,2.0,469.0
1,1.0,6,1,352.0,4.0,5.0,42.0,3.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,33.0,146.0,0.0,1.0,497.0
2,1.0,7,1,386.0,4.0,2.0,88.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,50.0,76.0,0.0,3.0,436.0
3,1.0,8,1,519.0,2.0,7.0,75.0,5.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,49.0,188.0,0.0,0.0,686.0
4,1.0,9,1,411.0,3.0,3.0,57.0,6.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,38.0,114.0,0.0,3.0,559.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,9.0,255,1,496.0,2.0,5.0,55.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,66.0,108.0,0.0,2.0,471.0
273,9.0,256,1,455.0,5.0,4.0,110.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,51.0,141.0,0.0,2.0,645.0
274,9.0,69A,1,323.0,3.0,1.0,53.0,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,37.0,146.0,0.0,2.0,529.0
275,9.0,81A,1,445.0,3.0,6.0,88.0,7.0,2.0,0.0,...,0.0,3.0,0.0,0.0,0.0,78.0,154.0,0.0,3.0,649.0


In [8]:
import geopandas
geodf = geopandas.read_file("elections-2024-bureaux-de-vote.geojson")
geodf = geodf.rename(columns={'uniq_bdv': 'bdv'})
geodf['bdv'] = geodf['bdv'].str.lstrip('0')

In [9]:
geodf

,geo_point_2d,geo_shape,bdv,nom,adresse,geometry
0,"{ ""lon"": 1.4441076863240256, ""lat"": 43.6032242...",None,1,Capitole – Salle des Commissions,Place du Capitole,"POLYGON ((1.44199 43.60309, 1.44243 43.60374, ..."
1,"{ ""lon"": 1.4518250536978159, ""lat"": 43.6097919...",None,14,École élémentaire Bayard,60 rue Matabiau,"POLYGON ((1.44917 43.60976, 1.45137 43.61097, ..."
2,"{ ""lon"": 1.4517296952361987, ""lat"": 43.6068044...",None,18,Salle Osète,6 rue du Lieutenant Colonnel Pélissier,"POLYGON ((1.44872 43.60577, 1.45132 43.60751, ..."
3,"{ ""lon"": 1.4543810487475033, ""lat"": 43.6179837...",None,22,École élémentaire Bonnefoy,18 rue du Faubourg Bonnefoy,"POLYGON ((1.45295 43.6198, 1.45302 43.61978, 1..."
4,"{ ""lon"": 1.4610570363388984, ""lat"": 43.6209217...",None,26,Espace Bonnefoy-Périole,25 rue de Périole,"POLYGON ((1.45788 43.62128, 1.45861 43.62104, ..."
...,...,...,...,...,...,...
272,"{ ""lon"": 1.4786871468984617, ""lat"": 43.6034030...",None,237,École élémentaire Château de l'Hers,8 rue Claudius Rougenet,"POLYGON ((1.47539 43.60458, 1.47571 43.60466, ..."
273,"{ ""lon"": 1.4851021473801029, ""lat"": 43.6009596...",None,238,École élémentaire Château de l'Hers,8 rue Claudius Rougenet,"POLYGON ((1.47599 43.60071, 1.47619 43.60158, ..."
274,"{ ""lon"": 1.4920940290582239, ""lat"": 43.5622340...",None,239,École élémentaire Henri Guillaumet,30 chemin de Bitet,"POLYGON ((1.46878 43.57466, 1.46967 43.57472, ..."
275,"{ ""lon"": 1.4830837050801509, ""lat"": 43.5771227...",None,241,École élémentaire Henri Guillaumet,30 chemin de Bitet,"POLYGON ((1.48029 43.57663, 1.48031 43.57799, ..."


In [25]:
combined_data = []
for election, df in adjusted_election_dataframes.items():
    df['election'] = election  # Ajouter la colonne 'election'
    combined_data.append(df)   # Ajouter le DataFrame à la liste

# Combiner tous les DataFrames en un seul
combined_df = pd.concat(combined_data, ignore_index=True)

In [10]:
df = adjusted_election_dataframes['EUR24']
df['bdv'] = df['bdv'].astype(str)
df_joined = df.merge(geodf, on='bdv', how='inner')

In [11]:
df_joined

,Circonscription,bdv,Tour,Abstentions,Alliance rurale,Asselineau,Besoin europe,Blancs,Changer europe,Democratie representative,...,RN,Reveiller europe,Unite nationale,Urgence revolution,Votants,geo_point_2d,geo_shape,nom,adresse,geometry
0,1.0,5,1,350.0,6.0,3.0,91.0,5.0,1.0,0.0,...,38.0,80.0,0.0,2.0,469.0,"{ ""lon"": 1.4373794149183166, ""lat"": 43.6062377...",None,Salle Barcelone,22 Allée de Barcelone,"POLYGON ((1.43381 43.60261, 1.43434 43.60371, ..."
1,1.0,6,1,352.0,4.0,5.0,42.0,3.0,1.0,0.0,...,33.0,146.0,0.0,1.0,497.0,"{ ""lon"": 1.4412013699993496, ""lat"": 43.6081662...",None,Salle Barcelone,22 Allée de Barcelone,"POLYGON ((1.43752 43.60849, 1.43875 43.60926, ..."
2,1.0,7,1,386.0,4.0,2.0,88.0,1.0,0.0,0.0,...,50.0,76.0,0.0,3.0,436.0,"{ ""lon"": 1.4440136456225348, ""lat"": 43.6065087...",None,Salle Barcelone,22 Allée de Barcelone,"POLYGON ((1.44567 43.60792, 1.44567 43.60654, ..."
3,1.0,8,1,519.0,2.0,7.0,75.0,5.0,1.0,0.0,...,49.0,188.0,0.0,0.0,686.0,"{ ""lon"": 1.4374293352559915, ""lat"": 43.6109152...",None,École élémentaire du Nord,13 boulevard d'Arcole,"POLYGON ((1.4367 43.61418, 1.43682 43.61408, 1..."
4,1.0,9,1,411.0,3.0,3.0,57.0,6.0,0.0,0.0,...,38.0,114.0,0.0,3.0,559.0,"{ ""lon"": 1.4416788028623433, ""lat"": 43.6101538...",None,École élémentaire du Nord,13 boulevard d'Arcole,"POLYGON ((1.44277 43.61095, 1.44295 43.61067, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9.0,255,1,496.0,2.0,5.0,55.0,2.0,0.0,0.0,...,66.0,108.0,0.0,2.0,471.0,"{ ""lon"": 1.4584710054115886, ""lat"": 43.5807446...",None,Groupe Scolaire Anatole France,21 boulevard de la Méditerranée,"POLYGON ((1.45776 43.58293, 1.45795 43.58278, ..."
272,9.0,256,1,455.0,5.0,4.0,110.0,0.0,0.0,0.0,...,51.0,141.0,0.0,2.0,645.0,"{ ""lon"": 1.4591834561605626, ""lat"": 43.5844968...",None,Groupe Scolaire Anatole France,21 boulevard de la Méditerranée,"POLYGON ((1.45513 43.58466, 1.45588 43.58566, ..."
273,9.0,69A,1,323.0,3.0,1.0,53.0,5.0,1.0,0.0,...,37.0,146.0,0.0,2.0,529.0,"{ ""lon"": 1.4503692890965008, ""lat"": 43.5820973...",None,Maison des Associations,3 place Guy Hersant,"POLYGON ((1.44769 43.58437, 1.44783 43.5844, 1..."
274,9.0,81A,1,445.0,3.0,6.0,88.0,7.0,2.0,0.0,...,78.0,154.0,0.0,3.0,649.0,"{ ""lon"": 1.4551221654082043, ""lat"": 43.5688909...",None,Gymnase Jean Moulin,7 avenue des Ecoles Jules Julien,"POLYGON ((1.45107 43.56948, 1.45393 43.5712, 1..."


In [12]:
df_joined['geo_point_2d'] = df_joined['geo_point_2d'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# Créer une géométrie de points à partir de lat/lon
# Extraire longitude et latitude depuis geo_point_2D
df_joined['lon'] = df_joined['geo_point_2d'].apply(lambda x: x['lon'] if pd.notnull(x) else None)
df_joined['lat'] = df_joined['geo_point_2d'].apply(lambda x: x['lat'] if pd.notnull(x) else None)
df_joined['geometry'] = df_joined.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Convertir le DataFrame en GeoDataFrame
gdf = GeoDataFrame(df_joined, geometry='geometry', crs="EPSG:4326")  # EPSG:4326 pour WGS 84

In [22]:
import folium

# Créer une carte interactive
m = gdf.explore(
    column='Abstentions',  # Colonne par défaut à afficher (remplacez par un nom valide)
    cmap='RdYlGn', #'viridis',          # Palette de couleurs
    marker_kwds=dict(radius=10, fill=True), # make marker radius 10px with fill
    legend=False, # show legend
    tooltip=False,
    popup=True,
    #tooltip=gdf.columns,     # Colonnes affichées en survol
    #popup=gdf.columns,       # Colonnes affichées en popup
    tiles=None, #'OpenStreetMap',   # Basemap
    save=True,               # Sauvegarde la carte
#    path='map_interactive.html'  # Nom du fichier HTML
)
folium.TileLayer('cartodbpositron', control=False).add_to(m)  # use folium to add alternative tiles
m.save("carte.html")
#gdf.to_csv("./results/resultats.csv")


In [ ]:
# CARTE AVEC CHOIX

from folium import Choropleth

# Initialiser la carte
m = folium.Map(location=[gdf['lat'].mean(), gdf['lon'].mean()], zoom_start=12, tiles='OpenStreetMap')

# Ajouter des calques pour chaque élection et chaque variable
elections = gdf['election_name'].unique()
variables = ['abstention', 'votants', 'blancs']  # Variables disponibles

for election in elections:
    for variable in variables:
        # Filtrer les données pour l'élection et la variable données
        subset = gdf[gdf['election_name'] == election]
        
        # Ajouter un calque choroplèthe
        Choropleth(
            geo_data=subset,
            name=f"{election} - {variable}",
            data=subset,
            columns=['bdv', variable],
            key_on='feature.properties.bdv',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f"{variable.capitalize()} pour {election}",
        ).add_to(m)

# Ajouter le contrôle des couches
folium.LayerControl().add_to(m)

from branca.element import Template, MacroElement

# Définir le code HTML des menus
template = """
{% macro html(this, kwargs) %}
<!doctype html>
<html>
<head>
    <script>
        function updateMap() {
            var election = document.getElementById("election-select").value;
            var variable = document.getElementById("variable-select").value;
            var layers = document.getElementsByClassName("leaflet-layer");
            
            for (var i = 0; i < layers.length; i++) {
                layers[i].style.display = "none";
            }
            
            var selectedLayer = document.querySelector(`[data-name="${election} - ${variable}"]`);
            if (selectedLayer) {
                selectedLayer.style.display = "block";
            }
        }
    </script>
</head>
<body>
    <div style="position: absolute; top: 10px; left: 10px; z-index: 9999; background: white; padding: 10px;">
        <label for="election-select">Élection :</label>
        <select id="election-select" onchange="updateMap()">
            {% for election in elections %}
            <option value="{{ election }}">{{ election }}</option>
            {% endfor %}
        </select>
        <br>
        <label for="variable-select">Variable :</label>
        <select id="variable-select" onchange="updateMap()">
            {% for variable in variables %}
            <option value="{{ variable }}">{{ variable }}</option>
            {% endfor %}
        </select>
    </div>
</body>
</html>
{% endmacro %}
"""

# Injecter les menus dans la carte
macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)

# Sauvegarder la carte
m.save('map_with_dropdowns.html')